### 품사 태깅

In [2]:
# csv파일 불러옴
import csv
import re

raw = []
f = open(r'C:\Users\0000\Desktop\sentenceExtraction_with_Article\ytn_article.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()  

# 빈 text 제거 & 특수문자 제거
text = []
for i in range(len(raw)):
    if(raw[i][0] is not ' '):
        newtext = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw[i][0])
        text.append(newtext)

In [4]:
# 품사 태깅(주어진 텍스트를 형태소 단위로 나누고 나눠진 형태소를 해당하는 품사와 함께 리스트화)
# stemming(단어들을 원형으로 포현), normalization(표현 방법이 다른 단어들을 통합시켜 같은 단어로 만듬)
import json
import os 
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc): # 형태소 분석기인 Okt를 이용해서 품사를 태깅시킴
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

os.chdir(r'C:\Users\0000\Desktop\sentenceExtraction_with_Article') # 경로 설정

C:\Users\0000\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [5]:
if os.path.isfile('text_docs.json'): # 경로에 text_docs.json 파일이 존재하면 json파일을 만들지 않고 load함
    with open('text_docs.json', encoding='utf-8') as f:
        text_docs = json.load(f)
else: # json파일이 없을 경우 text를 품사 태깅시킨 데이터로 변환시켜 json파일로 저장
    text_docs = [(tokenize(row)) for row in text]
    # JSON 파일로 저장
    with open('text_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(text_docs, make_file, ensure_ascii=False, indent="\t")

In [6]:
# 태깅된 각 형태소들을 하나의 list로 합침
tokens = [t for d in text_docs for t in d]
print(len(tokens))

1707035


In [7]:
import nltk
from pprint import pprint

nltk_text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
total_tokens = nltk_text.tokens
print(len(total_tokens))

# 중복을 제외한 토큰의 개수
set_total_tokens = list(set(nltk_text.tokens))
print(len(set_total_tokens))            

# 출현 빈도가 높은 상위 토큰 10개
#pprint(nltk_text.vocab().most_common(50))

1707035
24065


### Word2vec

In [8]:
# 빈도수가 50개 이상인 token 확인.
most_common_tokens = nltk_text.vocab().most_common(len(set_total_tokens))
for i in range(len(set_total_tokens)):
    if(most_common_tokens[i][1]<50):
        index = i
        break
most_common_tokens_50 = most_common_tokens[:index]

In [9]:
# Word embedding(Word2vec 사용)
from gensim.models import Word2Vec
embedding_model = Word2Vec(text_docs, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

C:\Users\0000\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
for s in set_total_tokens:
    if("피해" in s):
        print(s)

피해/Noun
피해자/Noun
피해망상/Noun


In [128]:
# 두 벡터 사이의 코사인 유사도를 구함. 값이 클수록 비슷한 단어
word_similar = embedding_model.most_similar(positive=["피해/Noun"], topn=10)
#print(embedding_model.most_similar(positive=["피해/Noun"], topn=100))
print([i[0] for i in word_similar])

['인명/Noun', '침수/Noun', '재산/Noun', '화재/Noun', '사고/Noun', '농작물/Noun', '사망자/Noun', '폐/Noun', '규모/Noun', '대처/Noun']


C:\Users\0000\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [129]:
words_damage = ['인명', '침수', '재산', '화재', '사고', '농작물', '사망자']

## 기사를 문장별로 토큰화

In [130]:
from konlpy.tag import Kkma
kkma = Kkma()

In [132]:
sentence_tokens = [] # 기사를 문장별로 토큰화한 결과값을 list에 저장
articles = list(set(text)) # 중복되는 기사 제거
for article in articles:
    sentence_tokens.append(kkma.sentences(article))    

## '피해'라는 단어와 top7 단어가 함께 들어있는 문장 추출

In [133]:
sentences_damage = []
for sentences in sentence_tokens:
    for sentence in sentences:
        isExist = False        
        for word_damage in words_damage:
            if isExist == False:
                if (word_damage in sentence) and ('피해' in sentence):
                    sentences_damage.append(sentence)
                    isExist = True

In [139]:
sentences_damage[10:20]

['또 일부 저지대 주택과 농경지가 침수되고 행인이 고립되는 등 10 여 건의 피해가 났습니다.',
 '온 열 질환자와 사망자가 급증하고 전력 소비량이 최대치를 경신하며 농작물과 가축 제조업 등에서 막대한 피해가 발생한다는 겁니다.',
 '다행히 사고 당시 다리를 지나던 차량이 많지 않아 피해는 발생하지 않았습니다.',
 '직원들이 재빨리 대피해 다친 사람은 없었지만 부화장 한 개 동이 모두 타 8천만 원의 재산피해가 났습니다.',
 '대전 통영 고속도로 하행선 경남 산청 IC 인근에서 그레이스 승합차에 불이 나 전소됐지만 인명 피해는 없었습니다.',
 '■ 고물 상 화재 …2 천만 원 피해 경남 김 해의 한 고물상에서도 불이 나 고물상 내 조립식 건물과 야적장 일부를 태워 소방서 추산 2천만 원 상당의 재산피해를 내고 20여 분 만에 꺼졌습니다.',
 '화재 당시 현장에 사람이 없어 인명피해는 없었습니다.',
 '[ 전 경 윤 침수 피해 횟집 주인 더위를 식히나 했지 저녁에는 그랬는데 난데없이 너무 많이 쏟아지니까 피해가 이만저만이 아니죠.]',
 '주택과 상가 침수 등 접수된 비 피해 신고는 강릉 지역에서만 80건이 넘습니다.',
 '이와 함께 불볕 더위로 전력 사용량이 크게 늘면서 축사 화재와 정전사고가 우려되므로 미리 점검해야 피해를 막을 수 있습니다.']

In [137]:
len(sentences_damage)

369

In [140]:
f = open(r'C:\Users\0000\Desktop\sentenceExtraction_with_Article\sentences_damage.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i in sentences_damage:
    wr.writerow([i])
f.close()